In [1]:
from datetime import datetime
from pathlib import Path
from typing import Any, Final, TypeAlias, cast, TypedDict

import cv2
import numpy as np
from cv2 import aruco
from cv2.typing import MatLike
from loguru import logger
from matplotlib import pyplot as plt
from numpy.typing import ArrayLike
from numpy.typing import NDArray as NDArrayT
import orjson

NDArray: TypeAlias = np.ndarray

In [2]:
INPUT_IMAGE = Path("merged_uv_layout.png")
# 7x7
DICTIONARY: Final[int] = aruco.DICT_7X7_1000
# 400mm
MARKER_LENGTH: Final[float] = 0.4

In [3]:
aruco_dict = aruco.getPredefinedDictionary(DICTIONARY)
detector = aruco.ArucoDetector(
    dictionary=aruco_dict, detectorParams=aruco.DetectorParameters()
)

In [4]:
frame = cv2.imread(str(INPUT_IMAGE))
grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# pylint: disable-next=unpacking-non-sequence
markers, ids, rejected = detector.detectMarkers(grey)

In [5]:
# Note: BGR
RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)
YELLOW = (0, 255, 255)
GREY = (128, 128, 128)
CYAN = (255, 255, 0)
MAGENTA = (255, 0, 255)
ORANGE = (0, 165, 255)
PINK = (147, 20, 255)

UI_SCALE = 10
UI_SCALE_FONT = 8
UI_SCALE_FONT_WEIGHT = 20

In [6]:
out = frame.copy()
# `markers` is [N, 1, 4, 2]
# `ids` is [N, 1]
if ids is not None:
    markers = np.reshape(markers, (-1, 4, 2))
    ids = np.reshape(ids, (-1, 1))
    # logger.info("markers={}, ids={}", np.array(markers).shape, np.array(ids).shape)
    for m, i in zip(markers, ids):
        # logger.info("id={}, center={}", i, center)
        center = np.mean(m, axis=0).astype(int) # type: ignore
        # BGR
        color_map = [RED, GREEN, BLUE, YELLOW]
        for color, corners in zip(color_map, m):
            corners = corners.astype(int)
            out = cv2.circle(out, corners, 5*UI_SCALE, color, -1)
        cv2.circle(out, tuple(center), 5*UI_SCALE, CYAN, -1)
        cv2.putText(
            out,
            str(i),
            tuple(center),
            cv2.FONT_HERSHEY_SIMPLEX,
            1*UI_SCALE_FONT,
            MAGENTA,
            UI_SCALE_FONT_WEIGHT,
        )

In [11]:
class Marker(TypedDict):
    id: int
    center: NDArray
    corners: NDArray


output_markers: list[Marker] = []
if ids is not None:
    IMAGE_WIDTH = frame.shape[1]
    IMAGE_HEIGHT = frame.shape[0]

    def normalize_point(point: NDArrayT[Any]) -> NDArrayT[np.float64]:
        """
        input could be: [N, 2] or [2]
        """
        if point.ndim == 1:
            return point / np.array([IMAGE_WIDTH, IMAGE_HEIGHT])
        elif point.ndim == 2:
            return point / np.array([IMAGE_WIDTH, IMAGE_HEIGHT])
        else:
            raise ValueError(f"Invalid shape: {point.shape}")

    def flip_y(point: NDArrayT[Any], y_max: int) -> NDArrayT[Any]:
        """
        flip y axis;

        Usually OpenCV image y-axis is inverted. (origin at top-left)
        In UV layout, the origin is at bottom-left.
        """
        return np.array([point[0], y_max - point[1]])

    for m, i in zip(markers, ids):
        center = np.mean(m, axis=0).astype(int)  # type: ignore
        output_markers.append(
            {
                "id": i[0],
                "center": flip_y(normalize_point(center), 1),
                "corners": np.array([flip_y(normalize_point(c), 1) for c in m]),
            }
        )

with open("output/aruco_3d_coords.json", "wb") as f:
    f.write(orjson.dumps(output_markers, option=orjson.OPT_SERIALIZE_NUMPY))

In [ ]:
plt.imshow(cv2.cvtColor(out, cv2.COLOR_BGR2RGB))

In [ ]:
cv2.imwrite("merged_uv_layout_with_markers.png", out)

In [10]:
from typing import Optional, Union

import numpy as np
import trimesh
from jaxtyping import Float, Int, Num, jaxtyped
from beartype import beartype

@jaxtyped(typechecker=beartype)
def interpolate_uvs_to_3d(
    uv_points: Num[NDArray, "N 2"],
    vertices: Num[NDArray, "V 3"],
    uvs: Num[NDArray, "V 2"],
    faces: Num[NDArray, "F 3"],
    epsilon: float = 1e-6,
) -> Num[NDArray, "N 3"]:
    """
    Map multiple UV points to 3D coordinates using barycentric interpolation.

    Args:
        uv_points: (N, 2) array of UV coordinates in [0,1]
        vertices: (V, 3) array of mesh vertex positions
        uvs:      (V, 2) array of per-vertex UV coordinates
        faces:    (F, 3) array of triangle vertex indices
        epsilon:  barycentric inside-triangle tolerance

    Returns:
        (N, 3) array of interpolated 3D coordinates (NaNs if no triangle found)
    """
    results = np.full((uv_points.shape[0], 3), np.nan, dtype=np.float64)

    for pi, uv_point in enumerate(uv_points):
        for face in faces:
            uv_tri = uvs[face]  # (3,2)
            v_tri = vertices[face]  # (3,3)

            A = np.array(
                [
                    [uv_tri[0, 0] - uv_tri[2, 0], uv_tri[1, 0] - uv_tri[2, 0]],
                    [uv_tri[0, 1] - uv_tri[2, 1], uv_tri[1, 1] - uv_tri[2, 1]],
                ]
            )
            b = uv_point - uv_tri[2]

            try:
                w0, w1 = np.linalg.solve(A, b)
                w2 = 1.0 - w0 - w1
                if min(w0, w1, w2) >= -epsilon:
                    results[pi] = w0 * v_tri[0] + w1 * v_tri[1] + w2 * v_tri[2]
                    break  # Stop after first matching triangle
            except np.linalg.LinAlgError:
                continue

    return results


@jaxtyped(typechecker=beartype)
def interpolate_uvs_to_3d_trimesh(
    uv_points: Num[NDArray, "N 2"],
    mesh: Union[trimesh.Trimesh, trimesh.Scene],
    epsilon: float = 1e-6,
) -> Num[NDArray, "N 3"]:
    """
    Wrapper for batched UV-to-3D interpolation using a trimesh mesh or scene.

    Args:
        uv_points: (N, 2) UV coordinates to convert
        mesh: a Trimesh or Scene object
        epsilon: barycentric epsilon tolerance

    Returns:
        (N, 3) array of 3D positions (NaN if outside mesh)
    """
    if isinstance(mesh, trimesh.Scene):
        if len(mesh.geometry) == 0:
            raise ValueError("Scene has no geometry.")
        mesh = list(mesh.geometry.values())[0]

    if not isinstance(mesh, trimesh.Trimesh):
        raise TypeError("Expected a Trimesh or Scene with geometry.")

    if mesh.visual is None:
        raise ValueError("Mesh does not have visual.")

    if mesh.visual.uv is None:
        raise ValueError("Mesh does not have UVs.")

    return interpolate_uvs_to_3d(
        uv_points=uv_points,
        vertices=mesh.vertices,
        uvs=mesh.visual.uv,
        faces=mesh.faces,
        epsilon=epsilon,
    )